# Visualizing processing steps for Sentinel-1 SAR imagery

2023-06-15

In [2]:

import geopandas as gpd
import pandas as pd
import ee
import folium
import shapely.geometry
from shapely.geometry import shape
from matplotlib import pyplot as plt

# ee.Authenticate()
ee.Initialize(project='gsl-monitoring')

## Read

In [104]:
gdf = gpd.read_file('../../data/definitions/preliminary-roi.geojson')
polygon_geojson = shapely.geometry.mapping(gdf.loc[0, 'geometry'])
roi = ee.Geometry.Polygon(polygon_geojson['coordinates'])

collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(roi) \
    .filterDate('2014-01-01', '2023-05-30') \
    .select(['VV', 'VH', 'angle'])

collection.size().getInfo()

image_ids = [
    # 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20180313T133325_20180313T133350_020997_0240CE_1D8D',
    'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20180313T133350_20180313T133415_020997_0240CE_ADB5',
    # 'COPERNICUS/S1_GRD/S1B_IW_GRDH_1SDV_20180309T011826_20180309T011851_009948_01205D_494A',
    'COPERNICUS/S1_GRD/S1B_IW_GRDH_1SDV_20170331T012629_20170331T012658_004946_008A59_DC5F',
    # 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20170404T134138_20170404T134203_015995_01A623_4791',
    # 'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20170404T134203_20170404T134228_015995_01A623_1DD4',
    'COPERNICUS/S1_GRD/S1A_IW_GRDH_1SDV_20230522T133414_20230522T133439_048647_05D9E0_17C0'
]

In [95]:
collection.first().bandNames().getInfo()

['VV', 'VH', 'angle']

## Function definitions

In [105]:
def get_image_collection(image_ids):
    def get_images(image_id):
        return ee.Image(image_id)

    images = ee.ImageCollection.fromImages(list(map(get_images, image_ids)))\
                                .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
                                .filter(ee.Filter.eq('instrumentMode', 'IW'))
    return images

def median_filter(img, size):
    # see here https://mbonnema.github.io/GoogleEarthEngine/07-SAR-Water-Classification/
    vv = img.select('VV')
    vv_smoothed = vv.focal_median(size,'circle','pixels').rename('VV_filtered')
    vh = img.select('VH')
    vh_smoothed = vh.focal_median(size,'circle','pixels').rename('VH_filtered')
    return img.addBands(vv_smoothed).addBands(vh_smoothed)


def convert_to_linear(image):
    return ee.Image(10).pow(image.divide(10))

def normalize(img):
    stats = img.reduceRegion(
        reducer = ee.Reducer.minMax(),
        geometry = roi,
        scale = 10,
        bestEffort = True)

    min_val_VV = ee.Number(stats.get('VV_filtered_min'))
    max_val_VV = ee.Number(stats.get('VV_filtered_max'))
    min_val_VH = ee.Number(stats.get('VH_filtered_min'))
    max_val_VH = ee.Number(stats.get('VH_filtered_max'))
    min_val_VH_VV = ee.Number(stats.get('VH_VV_filtered_min'))
    max_val_VH_VV = ee.Number(stats.get('VH_VV_filtered_max'))

    quantiles = img.reduceRegion(
        reducer = ee.Reducer.percentile([1,2,98,99]),
        geometry = roi,
        scale = 10,
        bestEffort = True)

    # print(quantiles.getInfo())

    # {'VH_VV_filtered_p1': 1.3707270989343514,
    #  'VH_VV_filtered_p5': 2.6272155209216566,
    #  'VH_VV_filtered_p95': 14.876791757593027,
    #  'VH_VV_filtered_p99': 18.867862562150677,
    #  'VH_filtered_p1': 0.0008323809960359647,
    #  'VH_filtered_p5': 0.0008323809960359647,
    #  'VH_filtered_p95': 0.027817527574252796,
    #  'VH_filtered_p99': 0.057139821900548365,
    #  'VV_filtered_p1': 0.006781440620247682,
    #  'VV_filtered_p5': 0.006781440620247682,
    #  'VV_filtered_p95': 0.1481321116306729,
    #  'VV_filtered_p99': 0.27378398892299255}

    q98_val_VV = ee.Number(quantiles.get('VV_filtered_p98'))
    q98_val_VH = ee.Number(quantiles.get('VH_filtered_p98'))

    image_vv = img.select('VV_filtered').clamp(0, q98_val_VV)
    image_vh = img.select('VH_filtered').clamp(0, q98_val_VH)

    image_vv = image_vv.subtract(min_val_VV).divide(q98_val_VV.subtract(min_val_VV)).rename('VV')
    image_vh = image_vh.subtract(min_val_VH).divide(q98_val_VH.subtract(min_val_VH)).rename('VH')
    image_vh_vv = img.select('VH_VV_filtered').subtract(min_val_VH_VV).divide(max_val_VH_VV.subtract(min_val_VH_VV)).rename('VH_VV')

    # print(image_vv.reduceRegion(ee.Reducer.minMax(), roi, 10, maxPixels=1e9).getInfo())
    img_normalized = image_vv.clamp(0, 1).addBands(image_vh.clamp(0, 1)).addBands(image_vh_vv.clamp(0, 1))
    # print(img_normalized.reduceRegion(ee.Reducer.minMax(), roi, 10, maxPixels=1e9).getInfo())
    # print(ee.Number(image_vv.reduceRegion(reducer=ee.Reducer.percentile([99]),geometry=roi,scale=10,bestEffort=True).get('VV_filtered_p99')).getInfo())

    return img_normalized

def show_map(img, roi, vis_params):
    def add_ee_layer(self, ee_image_object, vis_params, name):
        map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
        folium.raster_layers.TileLayer(
            tiles=map_id_dict['tile_fetcher'].url_format,
            attr="Map Data © Google Earth Engine",
            name=name,
            overlay=True,
            control=True
        ).add_to(self)
    folium.Map.add_ee_layer = add_ee_layer
    my_map = folium.Map(location=[roi.centroid().coordinates().get(1).getInfo(),
                                  roi.centroid().coordinates().get(0).getInfo()],
                        zoom_start=9)
    my_map.add_ee_layer(img, vis_params, 'Normalized SAR Image')
    my_map.add_child(folium.LayerControl())
    return my_map

In [106]:
collection_subset = get_image_collection(image_ids)

In [67]:
# abandoned (for now) implementation of a composite image

# vh = collection_subset.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
#     .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
#     .filter(ee.Filter.eq('instrumentMode', 'IW'))

# vhAscending = vh.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
# vhDescending = vh.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

# composite = ee.Image.cat([
#     vhAscending.select('VH').max(),
#     vhDescending.select('VH').max(),
#     ee.ImageCollection(vhAscending.select('VV').merge(vhDescending.select('VV'))).max(),
#     ]).focal_median()

# composite.getInfo()


## Process

Make linear, then apply speckle filter, then normalize

In [122]:
image = ee.Image(collection_subset.toList(collection_subset.size()).get(1)).clip(roi).select(['VV', 'VH', 'angle'])
image.getInfo()
# get system:index
print(image.get('system:index').getInfo())
# print(image.bandNames().getInfo(), image.reduceRegion(ee.Reducer.minMax(), roi, 10, maxPixels=1e9).getInfo())

S1B_IW_GRDH_1SDV_20170331T012629_20170331T012658_004946_008A59_DC5F


Visualize angle band distribution

In [72]:
print(image.bandNames().getInfo(), image.reduceRegion(ee.Reducer.minMax(), roi, 10, maxPixels=1e9).getInfo())
show_map(
    image,
    roi, 
    {
        'bands': ['angle'],
        'min': [35],
        'max': [44]
    }
)

['VV', 'VH', 'angle'] {'VH_max': 24.432217201862837, 'VH_min': -73.37154462911002, 'VV_max': 36.7585282396283, 'VV_min': -70.35610879910682, 'angle_max': 44.76335144042969, 'angle_min': 36.76759338378906}


Visualize raw VV band distribution

In [123]:
show_map(
    image,
    roi, 
    {
        'bands': ['VV'],
        'min': [-25],
        'max': [-5]
    }
)

### Make units linear

In [124]:
image_linear = convert_to_linear(image.select(['VV', 'VH']))
# print(image_linear.bandNames().getInfo(), image_linear.reduceRegion(ee.Reducer.minMax(), roi, 10, maxPixels=1e9).getInfo())

In [132]:
show_map(
    image_linear,
    roi, 
    {
        'bands': ['VV'],
        'min': [0],
        'max': [.075]
    }
)

### Apply speckle filter

In [133]:
image_filtered = median_filter(image_linear, 7)
# print(image_filtered.bandNames().getInfo(), image_filtered.reduceRegion(ee.Reducer.minMax(), roi, 10, maxPixels=1e9).getInfo())

In [135]:
show_map(
    image_filtered,
    roi, 
    {
        'bands': ['VV_filtered'],
        'min': [0],
        'max': [0.075]
    }
)

### Define ratio band

In [136]:
image_filtered = image_filtered.addBands(image_filtered.select('VV_filtered').divide(image_filtered.select('VH_filtered')).rename('VH_VV_filtered'))

### Normalize

In [137]:
image_normalized = normalize(image_filtered)

In [138]:
band = 'VV'
show_map(
    image_normalized.select([band]), 
    roi, 
    {
        'bands': [band],
        'min': [0],
        'max': [.1]
    }
)

In [139]:
band = 'VH'
show_map(
    image_normalized.select([band]), 
    roi, 
    {
        'bands': [band],
        'min': [0],
        'max': [.1]
    }
)


In [140]:
band = 'VH_VV'
show_map(
    image_normalized.select([band]), 
    roi, 
    {
        'bands': [band],
        'min': [0],
        'max': [.2]
    }
)

In [141]:
bands = ['VV', 'VH']
show_map(
    image_normalized.select(bands),
    roi, 
    {
        'bands': bands,
        'min': [0, 0],
        'max': [.1, .1]
    }
)

In [142]:
bands=['VV', 'VH', 'VH_VV']
show_map(
    image_normalized.select(bands),
    roi, 
    {
        'bands': bands,
        'min': [0, 0, 0],
        'max': [.1, .1, .2]
    }
)

## Make bands consistent type

In [200]:
# make all bands float64
image_normalized = image_normalized.select(['VV', 'VH', 'VH_VV']).float()

## Reproject bands to same CRS

In [18]:
# reproject each band individually
# print(image_normalized.getInfo())
print(image_normalized.projection().getInfo())
image_normalized = image_normalized.reproject('EPSG:6341', scale=image_normalized.projection().nominalScale())
print(image_normalized.projection().getInfo())
# image_normalized_vv = image_normalized.select(['VV']).reproject(crs='EPSG:6341', scale=image_normalized.projection().nominalScale())
# image_normalized_vh = image_normalized.select(['VH']).reproject(crs='EPSG:6341', scale=image_normalized.projection().nominalScale())
# image_normalized_vh_vv = image_normalized.select(['VH_VV']).reproject(crs='EPSG:6341', scale=image_normalized.projection().nominalScale())
# image_normalized_angle = image_normalized.select(['angle']).reproject(crs='EPSG:6341', scale=image_normalized.projection().nominalScale())
# # image_reprojected = image_normalized_vv.addBands(image_normalized_vh).addBands(image_normalized_vh_vv).addBands(image_normalized_angle)
# print(image_normalized_vv.getInfo())
# print(image_normalized_vh.getInfo())
# print(image_normalized_vh_vv.getInfo())
# print(image_normalized_angle.getInfo())

{'type': 'Projection', 'crs': 'EPSG:32612', 'transform': [10, 0, 265363.8038094812, 0, -10, 4661937.729569972]}
{'type': 'Projection', 'crs': 'EPSG:6341', 'transform': [10, 0, 0, 0, -10, 0]}


## Export

In [19]:
task = ee.batch.Export.image.toDrive(
    image=image_normalized,
    description=image.get('system:index').getInfo(),
    # scale=10, # if not provided, the native resolution of the image will be used
    maxPixels=1e13,
    region=roi.getInfo()['coordinates'],
    fileFormat='GeoTIFF',
    formatOptions={'cloudOptimized': True}
)

task.start()
task.status()

{'state': 'READY',
 'description': 'S1A_IW_GRDH_1SSV_20161025T133344_20161025T133409_013647_015DFD_D6D4',
 'creation_timestamp_ms': 1686840200863,
 'update_timestamp_ms': 1686840200863,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'ARTSLINRDGRD6MORPFGORTIV',
 'name': 'projects/gsl-monitoring/operations/ARTSLINRDGRD6MORPFGORTIV'}

In [20]:
task.status()

{'state': 'READY',
 'description': 'S1A_IW_GRDH_1SSV_20161025T133344_20161025T133409_013647_015DFD_D6D4',
 'creation_timestamp_ms': 1686840200863,
 'update_timestamp_ms': 1686840200863,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'ARTSLINRDGRD6MORPFGORTIV',
 'name': 'projects/gsl-monitoring/operations/ARTSLINRDGRD6MORPFGORTIV'}